In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from scipy.sparse import hstack
import pickle
from tqdm import tqdm
import pickle

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import time

physical_devices = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)


In [3]:
physical_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, SimpleRNN, Bidirectional, Embedding, Dropout, Input, GRU
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.feature_extraction.text import TfidfVectorizer
from keras import regularizers

In [5]:
cd ..

/home/harsh/projects/sembly-round2-harsh/task-1


In [6]:
cd ..

/home/harsh/projects/sembly-round2-harsh


In [7]:
df=pd.read_csv("task-1/data/cleaned_data.csv")
y_true=df['is_duplicate']

In [8]:
# transformer=pickle.load(open("task-1/data/tfidf/tfidf_transformer.pkl", "rb"))

In [9]:
question1=df["question1"][:10000]
question2=df["question2"][:10000]

In [10]:
all_questions = list(question1) + list(question2)
len(all_questions)

20000

In [11]:
vectorizer = TfidfVectorizer()
# analyze = vectorizer.build_analyzer()
transformer=vectorizer.fit(all_questions)

In [12]:
question1_vectors=transformer.transform(question1)
question2_vectors=transformer.transform(question2)

In [13]:
question1_vectors = question1_vectors.todense()
question2_vectors = question2_vectors.todense()

In [14]:
question1_vectors[5].shape

(1, 14711)

In [15]:
x=np.hstack((question1_vectors[:],question2_vectors[:]))
x.shape

(10000, 29422)

In [16]:
print(max(x[1][0]))

[[0. 0. 0. ... 0. 0. 0.]]


In [17]:
x_train,x_test,y_train,y_test = train_test_split(x,y_true[:10000],test_size=0.3,random_state=20)

In [18]:
model=Sequential()
model.add(Input(29422))
# model.add(SimpleRNN(8,activation="relu",input_shape=(29422,1,)))
# 
# model.add(Dropout(0.2))
# model.add(Dense(128,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(64,activation="relu"))
# model.add(Dense(64,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(32,activation="relu",kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(1,activation="sigmoid"))

In [19]:
callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_Accuracy',
            min_delta=1e-5,
            patience=10,
            verbose=1
        ),
        tf.keras.callbacks.ModelCheckpoint(
            filepath="task-1/saved_models/tfidf/gru",
            monitor='val_loss', 
            mode='min', 
            save_best_only=True,
            verbose=1
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss', 
            factor=0.2,
            patience=4, 
            min_lr=0.001)
    
]

# optimizer = tf.keras.optimizers.Adam(1e-5)
# loss = tf.keras.losses.BinaryCrossentropy()
# acc = tf.keras.metrics.Accuracy()

In [20]:
model.compile(loss = "binary_crossentropy", optimizer = "Adam", metrics = ["Accuracy"])

In [21]:
history = model.fit(x_train, y_train, epochs = 10, batch_size = 32, validation_data= (x_test, y_test), callbacks=callbacks, verbose=1)

Epoch 1/10
208/219 [===========================>..] - ETA: 0s - loss: 0.8005 - Accuracy: 0.6318
Epoch 1: val_loss improved from inf to 0.63745, saving model to task-1/saved_models/tfidf/gru
INFO:tensorflow:Assets written to: task-1/saved_models/tfidf/gru/assets
219/219 [==============================] - 3s 10ms/step - loss: 0.7924 - Accuracy: 0.6350 - val_loss: 0.6375 - val_Accuracy: 0.6843 - lr: 0.0010
Epoch 2/10
219/219 [==============================] - ETA: 0s - loss: 0.5375 - Accuracy: 0.7606
Epoch 2: val_loss improved from 0.63745 to 0.60315, saving model to task-1/saved_models/tfidf/gru
INFO:tensorflow:Assets written to: task-1/saved_models/tfidf/gru/assets
219/219 [==============================] - 2s 7ms/step - loss: 0.5375 - Accuracy: 0.7606 - val_loss: 0.6031 - val_Accuracy: 0.7243 - lr: 0.0010
Epoch 3/10
215/219 [============================>.] - ETA: 0s - loss: 0.3864 - Accuracy: 0.8555
Epoch 3: val_loss did not improve from 0.60315
219/219 [==============================]